# Scraping Team Articles

In [1]:
import requests
from bs4 import BeautifulSoup

In [59]:
def get_soup_from_url(url):
    html = requests.get(url).text
    return BeautifulSoup(html, 'html5lib')

In [3]:
ROOT_TEAM_URL = 'https://www.cbssports.com/college-basketball/news/2019-march-madness-bracket-field-ranking-all-68-teams-in-the-ncaa-tournament-bracket/'
root_soup = get_soup_from_url(ROOT_TEAM_URL)

https://www.cbssports.com/college-basketball/news/2019-march-madness-bracket-field-ranking-all-68-teams-in-the-ncaa-tournament-bracket/


In [17]:
team_divs = root_soup.find_all('div', class_='team-name')
names = []
article_urls = []
for div in team_divs:
    name = div.a.text.strip()
    articles_url = div.a['href']
    names.append(name)
    article_urls.append(articles_url)

In [54]:
import re

def googleize(name):
    name = re.sub('\s+', '+', name)
    name = name.lower()
    name = re.sub('[\.\,\-]', '', name)
    return f"https://www.google.com/search?q={name}+basketball"

In [60]:
import urllib
import re
import time

from tqdm import tqdm

article_lists = []
for name in tqdm(names):
    top_15 = []
    search_url = googleize(name)
    html = get_soup_from_url(search_url)
    filtered_anchors = [a for a in html.find_all('a') if re.search('http', a['href']) and not re.search('google', a['href'])]
    for a in filtered_anchors[:15]:
        match = re.search('http.+?&', a['href'])
        if match:
            top_15.append(match[0][:-1])
    article_lists.append(top_15)
    time.sleep(5)
    

100%|██████████| 68/68 [06:41<00:00,  5.82s/it]


In [62]:
import pickle

with open('article_url_lists.pkl', 'wb') as out:
    pickle.dump(article_lists, out)

In [67]:
with open('names.pkl', 'wb') as out:
    pickle.dump(names, out)

In [63]:
import pickle

with open('article_url_lists.pkl', 'rb') as i:
    lists = pickle.load(i)
    
with open('names.pkl', 'rb') as i:
    names = pickle.load(i)

In [69]:
import re

bracketed_stuff = re.compile("<.+?>")

def get_text_from_article_url(url):
    text_parts = []
    html = get_soup_from_url(url)
    paragraphs = html.find_all('p')
    for p in paragraphs:
        raw_text = p.text
        text_parts.append(bracketed_stuff.sub('', raw_text))
    return ' '.join(text_parts)
        

In [83]:
import time

from tqdm import tqdm

text_lists = []
error_urls = []
for url_list in tqdm(lists):
    texts = []
    for url in url_list:
        try:
            texts.append(get_text_from_article_url(url))
        except Exception as e:
            error_urls.append((url, e))
        time.sleep(1)
    text_lists.append(texts)






  0%|          | 0/68 [00:00<?, ?it/s]




  1%|▏         | 1/68 [00:22<25:22, 22.72s/it]




  3%|▎         | 2/68 [00:44<24:50, 22.58s/it]




  4%|▍         | 3/68 [01:11<25:46, 23.79s/it]




  6%|▌         | 4/68 [01:31<24:07, 22.61s/it]




  7%|▋         | 5/68 [01:54<23:49, 22.69s/it]




  9%|▉         | 6/68 [02:17<23:31, 22.77s/it]




 10%|█         | 7/68 [02:51<26:32, 26.11s/it]




 12%|█▏        | 8/68 [03:12<24:39, 24.66s/it]




 13%|█▎        | 9/68 [03:36<24:03, 24.46s/it]




 15%|█▍        | 10/68 [04:00<23:31, 24.34s/it]




 16%|█▌        | 11/68 [04:37<26:37, 28.02s/it]




 18%|█▊        | 12/68 [05:03<25:47, 27.64s/it]




 19%|█▉        | 13/68 [05:26<23:59, 26.17s/it]




 21%|██        | 14/68 [05:55<24:10, 26.86s/it]




 22%|██▏       | 15/68 [06:29<25:41, 29.08s/it]




 24%|██▎       | 16/68 [06:52<23:38, 27.27s/it]




 25%|██▌       | 17/68 [07:22<23:58, 28.20s/it]




 26%|██▋       | 18/68 [08:28<32:46, 39.33s/it]




 28%|██▊       | 19/68 [0

In [84]:
with open('text_lists.pkl', 'wb') as out:
    pickle.dump(text_lists, out)